In [ ]:
#enble gpu for faster model training or u can load the trained model simply

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!unzip '/content/drive/MyDrive/GdriveXbot/Indus_OS_Participants_Data.zip'

Archive:  /content/drive/MyDrive/GdriveXbot/Indus_OS_Participants_Data.zip
   creating: Indus_OS_Participants_Data/
  inflating: __MACOSX/._Indus_OS_Participants_Data  
  inflating: Indus_OS_Participants_Data/IndusOS_Data Dictionary.csv  
  inflating: __MACOSX/Indus_OS_Participants_Data/._IndusOS_Data Dictionary.csv  
  inflating: Indus_OS_Participants_Data/app_installs.csv  
  inflating: __MACOSX/Indus_OS_Participants_Data/._app_installs.csv  
  inflating: Indus_OS_Participants_Data/.DS_Store  
  inflating: __MACOSX/Indus_OS_Participants_Data/._.DS_Store  
  inflating: Indus_OS_Participants_Data/validation_data.csv  
  inflating: __MACOSX/Indus_OS_Participants_Data/._validation_data.csv  
  inflating: Indus_OS_Participants_Data/actual_set.csv  
  inflating: __MACOSX/Indus_OS_Participants_Data/._actual_set.csv  
  inflating: Indus_OS_Participants_Data/app_metadata.csv  
  inflating: __MACOSX/Indus_OS_Participants_Data/._app_metadata.csv  
  inflating: Indus_OS_Participants_Data/app_usa

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from lightgbm import LGBMRegressor
import xgboost as xgb
import datetime
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
dic=pd.read_csv('/content/Indus_OS_Participants_Data/IndusOS_Data Dictionary.csv')
app_ins=pd.read_csv('/content/Indus_OS_Participants_Data/app_installs.csv')
app_data=pd.read_csv('/content/Indus_OS_Participants_Data/app_metadata.csv')
app_u=pd.read_csv('/content/Indus_OS_Participants_Data/app_usage.csv')
ss=pd.read_csv('/content/Indus_OS_Participants_Data/sample_submission.csv')
u_data=pd.read_csv('/content/Indus_OS_Participants_Data/user_metadata.csv')
val=pd.read_csv('/content/Indus_OS_Participants_Data/validation_data.csv')
aa=pd.read_csv('/content/Indus_OS_Participants_Data/actual_set.csv')

In [4]:
app_u = app_u.loc[app_u['time_spent'] > 10000] 

#Converting milliseconds to minutes
app_u['time_spent'] = app_u['time_spent'].div(60000).round(2)

In [5]:
#removing repeted uid replace time_spent by its mean
app_u_new=app_u.groupby(['uid','item_id'])['time_spent'].agg(['mean'])
app_u_new=app_u_new.reset_index(drop=False)

In [6]:
app_u_new.head()

,uid,item_id,mean
0,2,5526,20.715000
1,2,20875,2.146667
2,2,25349,6.910000
3,2,29763,0.360000
4,2,145082,1.343333


In [7]:
#expermenting for getting a item_ids(apps) installed by a given user(uid)
#getting all instlled apps
app_ins_on = app_ins[app_ins.status!='uninstalled']
app_ins_on =app_ins_on.reset_index(drop=True)
app_ins_on.head()

,uid,item_id,status,install_date
0,281282,930055,installed,2022-03-05
1,88361,930055,installed,2022-07-09
2,79510,930055,installed,2022-05-14
3,53460,930055,installed,2022-07-16
4,221363,604992,installed,2022-03-14


In [8]:
#expermenting ,combining it with other item_id
app_ins_on_2 = app_ins_on[['uid','item_id']]
app_u_new_2 = app_u_new[['uid','item_id']]

full_new = pd.concat([app_u_new_2,app_ins_on_2],axis=0)
full_new =full_new.reset_index(drop=True)
full_new.head()

,uid,item_id
0,2,5526
1,2,20875
2,2,25349
3,2,29763
4,2,145082


In [9]:
#expermenting
full_all=full_new.groupby(['uid','item_id'])['item_id'].agg(['count'])
full_all=full_all.reset_index(drop=False)
full_all.head()

,uid,item_id,count
0,2,5526,1
1,2,20875,1
2,2,25349,1
3,2,29763,1
4,2,145082,1


In [10]:
# getting mean time_spent by user in mobile
dd=app_u.groupby(['uid'])['time_spent'].agg(['mean'])
dd = dd.reset_index(drop=False)
dd.head()

,uid,mean
0,2,14.015000
1,3,89.058333
2,4,42.490000
3,6,41.713214
4,7,14.014578


In [11]:
#we are combining app_user dataframe with actual dataset(given for validation)
#we can also use only app_user dataframe for training
#taking data given for validation adding it with training data for that we are constructing 
#an rating column for val data thta will be the mean rating of the user
aa_new = aa.copy()
aa_new = pd.merge(aa_new,dd,on=['uid'])
aa_new.head()

,uid,item_id,mean
0,177489,601235,24.774000
1,177489,824210,24.774000
2,177489,824963,24.774000
3,14644,64855,9.197857
4,14644,818840,9.197857


In [12]:
#adding it to the org data
full_one = pd.concat([aa_new,app_u_new],axis=0)
full_one=full_one.reset_index(drop=True)
full_one.head()

,uid,item_id,mean
0,177489,601235,24.774000
1,177489,824210,24.774000
2,177489,824963,24.774000
3,14644,64855,9.197857
4,14644,818840,9.197857


In [ ]:
#app_u_new=app_u_new.groupby(['uid','item_id'])['mean'].agg(['mean'])
#app_u_new=app_u_new.reset_index(drop=False)

In [ ]:
#app_u_new.head()

In [13]:
print('Number of unique books: ', app_u['item_id'].nunique())
print('Number of unique users: ', app_u['uid'].nunique())

Number of unique books:  47052
Number of unique users:  238036


In [14]:
print('Number of unique books: ', full_one['item_id'].nunique())
print('Number of unique users: ', full_one['uid'].nunique())

Number of unique books:  54585
Number of unique users:  238036


In [15]:
app_count = (full_one.
     groupby(by = ['item_id'])['uid'].
     count().
     reset_index().
     rename(columns = {'uid': 'app_count'})
     [['item_id', 'app_count']]
    )
app_count.head()

,item_id,app_count
0,234,4
1,332,2
2,380,1
3,381,4
4,399,1


In [16]:
rating_with_totalRatingCount = full_one.merge(app_count, left_on = 'item_id', right_on = 'item_id', how = 'left')
rating_with_totalRatingCount.head()

,uid,item_id,mean,app_count
0,177489,601235,24.774000,11
1,177489,824210,24.774000,10
2,177489,824963,24.774000,268
3,14644,64855,9.197857,3
4,14644,818840,9.197857,7549


In [17]:
popularity_threshold = 30
rating_popular_movie= rating_with_totalRatingCount.query('app_count >= @popularity_threshold')
rating_popular_movie.head()

,uid,item_id,mean,app_count
2,177489,824963,24.774000,268
4,14644,818840,9.197857,7549
5,14644,820362,9.197857,11031
7,14699,55857,4.494286,45074
8,14699,685639,4.494286,22106


In [18]:
user_count=(full_one.
     groupby(by = ['uid'])['item_id'].
     count().
     reset_index().
     rename(columns = {'item_id': 'total_user_Count'})
     [['uid', 'total_user_Count']]
    )
user_count.head()

,uid,total_user_Count
0,2,11
1,3,7
2,4,20
3,6,13
4,7,26


In [19]:
df = rating_popular_movie.merge(user_count, left_on = 'uid', right_on = 'uid', how = 'left')
df.head()

,uid,item_id,mean,app_count,total_user_Count
0,177489,824963,24.774000,268,13
1,14644,818840,9.197857,7549,7
2,14644,820362,9.197857,11031,7
3,14699,55857,4.494286,45074,7
4,14699,685639,4.494286,22106,7


In [20]:
#to include max users
user_threshold = 2
df= df.query('total_user_Count >= @user_threshold')
df.head()

,uid,item_id,mean,app_count,total_user_Count
0,177489,824963,24.774000,268,13
1,14644,818840,9.197857,7549,7
2,14644,820362,9.197857,11031,7
3,14699,55857,4.494286,45074,7
4,14699,685639,4.494286,22106,7


In [21]:
print('Number of unique books: ', df['item_id'].nunique())
print('Number of unique users: ', df['uid'].nunique())

Number of unique books:  4875
Number of unique users:  235108


In [22]:
# only select uid in test dataset(validation dataset) for avoiding memory overload
df_n=df[df.uid.isin(val['uid']) == True]

In [23]:
df_n.head()

,uid,item_id,mean,app_count,total_user_Count
285823,11,5526,33.500000,95316,21
285824,11,20875,19.830000,218222,21
285825,11,28125,15.096667,3237,21
285826,11,28297,0.750000,21417,21
285827,11,28320,25.545000,1378,21


In [24]:
df_n.head()

,uid,item_id,mean,app_count,total_user_Count
285823,11,5526,33.500000,95316,21
285824,11,20875,19.830000,218222,21
285825,11,28125,15.096667,3237,21
285826,11,28297,0.750000,21417,21
285827,11,28320,25.545000,1378,21


In [25]:
print('Number of unique books: ', df_n['item_id'].nunique())
print('Number of unique users: ', df_n['uid'].nunique())

Number of unique books:  4837
Number of unique users:  41300


In [26]:
from fastai.collab import *
from fastai.tabular.all import *

In [27]:
df_use = df_n[['uid','item_id','mean']]
df_use.columns = ['uid','item_id','rating']
df_use = df_use.reset_index(drop=True)
df_use.head(10)

,uid,item_id,rating
0,11,5526,33.500000
1,11,20875,19.830000
2,11,28125,15.096667
3,11,28297,0.750000
4,11,28320,25.545000
5,11,65649,22.710000
6,11,73395,12.642500
7,11,145569,3.460000
8,11,146048,1.720000
9,11,149528,2.890000


In [28]:
#fastai dataframe
dls = CollabDataLoaders.from_df(df_use, item_name='item_id', bs=64)
dls.show_batch()

,uid,item_id,rating
0,281116,69226,2.850000
1,261826,146530,3.340000
2,269107,5526,34.486000
3,95468,145082,15.572857
4,133851,69902,0.380000
5,295307,823255,7.760000
6,62274,27412,0.320000
7,92122,783441,0.230000
8,184976,589482,10.936666
9,293663,55843,0.230000


In [29]:
n_users  = len(dls.classes['uid'])
n_movies = len(dls.classes['item_id'])
n_factors = 5 #this factor is n.o of values used in an embedding to represent a user or apps 
#u can change the value to 20,32..etc

user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)

In [30]:
user_factors[0]

tensor([ 0.2025, -1.2399,  0.5821,  0.3340, -0.5072])

# first model

In [31]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
        
    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        return (users * movies).sum(dim=1)

In [ ]:
#use gpu for model training
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,1299.921875,1218.756958,03:51
1,1106.554565,1138.664185,04:19
2,1165.675293,1120.455078,04:10
3,1068.473999,1111.542847,03:44
4,1195.200684,1111.568359,03:24


In [ ]:
#saving the model for future use
#if the notebook restats u can simply load the model(no need of training it again)
learn.export('recommedation_1.pkl')

In [ ]:
#code for loading 
#learn = load_learner('/content/recommedation.pkl') #cpu=False)

In [ ]:
tst_dl = learn.dls.test_dl(aa)
preds,_=learn.get_preds(dl=tst_dl)
aa['rating'] = preds

In [ ]:
#validation prediction is too baddd ,but still the public leaderboard score was good
aa.head()

,uid,item_id,rating
0,177489,601235,0.000001
1,14644,64855,0.000001
2,14699,64855,0.000001
3,190856,64855,0.000001
4,70394,64866,0.000001


In [ ]:
aa.describe()

,uid,item_id,rating
count,326246.000000,326246.000000,326246.000000
mean,149584.321006,333137.368385,0.006799
std,87192.886382,325103.919681,0.024046
min,2.000000,332.000000,-0.071653
25%,73690.000000,68546.000000,-0.000035
50%,149333.000000,150714.000000,0.002550
75%,224654.500000,661784.000000,0.014026
max,301349.000000,930951.000000,0.076295


In [ ]:
del tst_dl
del preds
del ss

In [32]:
df_n_new = df_n[df_n.app_count>70]

In [33]:
md = pd.DataFrame()
md['item_id'] = df_n_new.item_id.unique() 
md.head()

,item_id
0,5526
1,20875
2,28125
3,28297
4,28320


In [34]:
ty = df_n.uid.unique()
len(ty)

41300

In [35]:
rt = md['item_id'].unique()
len(rt)

2685

In [ ]:
rt = md['item_id'].unique()
len(rt)

2685

first 20000 uids prediction

In [ ]:
#if u have ram then simply run this other than halfing ty for creating test dataframe.

    """"ss_uid = []
    ss_item = []
    for i in ty[0:20000]:
      for j in rt:
        ss_uid.append(i)
        ss_item.append(j)

In [ ]:
#we can prepare test dataset by avoiding the apps which are curremtly used by the user.
  """ss_uid = []
  ss_item = []
  for i in ty[10000:22000]:
    dd = all_ins[all_ins.uid==i]
    dd = dd.item_id.values
    cc=md[md.item_id.isin(dd) == False]
    cc = cc.item_id.unique()
    for j in cc:
      ss_uid.append(i)
      ss_item.append(j)

In [ ]:
#prepraing testdataset ,test dataframe by one uid to 2124 item_ids
#some times memory will runout .keep that in mind
ss_uid = []
ss_item = []
for i in ty[0:20000]:
  for j in rt:
    ss_uid.append(i)
    ss_item.append(j)

In [ ]:
nn_new=pd.DataFrame()
nn_new['uid']=ss_uid
nn_new['item_id']=ss_item
del ss_uid #cleraring ram memory
del ss_item
nn_new.head()

,uid,item_id
0,11,5526
1,11,20875
2,11,28125
3,11,28297
4,11,28320


In [ ]:
nn_new.head(10)

,uid,item_id
0,11,824963
1,11,818840
2,11,820362
3,11,55857
4,11,685639
5,11,15008
6,11,119002
7,11,821678
8,11,150270
9,11,636703


In [ ]:
#conversion of dataframe and prediction,takes more than 30 minutes

tst_dl = learn.dls.test_dl(nn_new)
preds,_=learn.get_preds(dl=tst_dl)
nn_new['rating'] = preds
del tst_dl,preds

In [ ]:
nn_new.head()

,uid,item_id,rating
0,11,824963,1.101398
1,11,818840,18.320621
2,11,820362,7.922470
3,11,55857,18.175312
4,11,685639,31.647732


In [ ]:
#getting 4 top apps with high rating ,all the recommeded apps are already in use
c_p = nn_new.set_index(['item_id']).groupby('uid')['rating'].nlargest(4).reset_index() 
del nn_new #avoid memory prb
c_p.head()

,uid,item_id,rating
0,11,25349,43.548767
1,11,25347,43.042652
2,11,149255,33.849594
3,11,62907,33.021606
4,15,25349,50.271683


In [ ]:
f1 = c_p[['uid','item_id']]
del c_p

f1.to_csv('f_20.csv',index=False)
f1.head(10)

,uid,item_id
0,11,25349
1,11,25347
2,11,149255
3,11,62907
4,15,25349
5,15,25347
6,15,20875
7,15,62907
8,18,25349
9,18,25347


In [ ]:
f1.item_id.value_counts()

second part

In [ ]:
ss_uid = []
ss_item = []
for i in ty[20000:]:
  for j in rt:
    ss_uid.append(i)
    ss_item.append(j)

In [ ]:
nn_new=pd.DataFrame()
nn_new['uid']=ss_uid
nn_new['item_id']=ss_item
del ss_uid #cleraring ram memory
del ss_item
nn_new.head(10)

,uid,item_id
0,146116,824963
1,146116,818840
2,146116,820362
3,146116,55857
4,146116,685639
5,146116,15008
6,146116,119002
7,146116,821678
8,146116,150270
9,146116,636703


In [ ]:
#second part of prediction takes around  30 minutes
tst_dl = learn.dls.test_dl(nn_new)
preds,_=learn.get_preds(dl=tst_dl)
nn_new['rating'] = preds
del tst_dl,preds

In [ ]:
c_p = nn_new.set_index(['item_id']).groupby('uid')['rating'].nlargest(4).reset_index() 
del nn_new #avoid memory prb
c_p.head()

,uid,item_id,rating
0,146116,25349,11.507840
1,146116,25347,10.842382
2,146116,62907,8.427426
3,146116,20875,8.115319
4,146117,25349,25.339376


In [ ]:
f2 = c_p[['uid','item_id']]
del c_p

f2.to_csv('f2_20.csv',index=False)
f2.head(10)

,uid,item_id
0,146116,25349
1,146116,25347
2,146116,62907
3,146116,20875
4,146117,25349
5,146117,25347
6,146117,20875
7,146117,62907
8,146122,25349
9,146122,25347


combining two dataframe with remaining uids

In [ ]:
t1 = pd.concat([f1,f2],axis=0)
t1 = t1.reset_index(drop=True)
t1.shape

(165200, 2)

In [ ]:
42252*4

169008

In [ ]:
# the desired length of pred dataframe is 169008

In [ ]:
ty_new = val[val.uid.isin(ty) == False]
ty_new =ty_new.uid
len(ty_new)

952

In [ ]:
rt=[25349, 55857, 20875, 145569]
ss_uid = []
ss_item = []
for i in ty_new:
  for j in rt:
    ss_uid.append(i)
    ss_item.append(j)
nn_new=pd.DataFrame()
nn_new['uid']=ss_uid
nn_new['item_id']=ss_item
del ss_uid #cleraring ram memory
del ss_item
nn_new.head(10)

,uid,item_id
0,14439,25349
1,14439,55857
2,14439,20875
3,14439,145569
4,14601,25349
5,14601,55857
6,14601,20875
7,14601,145569
8,79171,25349
9,79171,55857


In [ ]:
t2 =  pd.concat([t1,nn_new],axis=0)
t2 = t2.reset_index(drop=True)


(165200, 2)

In [ ]:
t2.shape

(169008, 2)

In [ ]:
t2.to_csv('final_11.csv',index=False)

#other methods used

https://github.com/data2450/Hackathons/blob/main/apps_recommedation_system/Hackathon_IndusOS_modified%20(2).ipynb

ure starter notebook modified for inference.

#**second model**

In [ ]:
class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0.17,1831.57)):
    #def __init__(self, n_users, n_movies, n_factors):
        self.user_factors = Embedding(n_users, n_factors)
        self.user_bias = Embedding(n_users, 1)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.movie_bias = Embedding(n_movies, 1)
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        res = (users * movies).sum(dim=1, keepdim=True)
        res += self.user_bias(x[:,0]) + self.movie_bias(x[:,1])
        return sigmoid_range(res, *self.y_range)

In [ ]:
model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,60591.003906,57611.214844,04:02
1,40214.445312,37786.609375,04:43
2,31227.404297,32588.664062,04:42
3,30178.511719,29475.408203,04:10
4,24102.458984,28958.074219,03:48


In [ ]:
learn.export('recommedation_2.pkl')

In [ ]:
tst_dl = learn.dls.test_dl(aa)
preds,_=learn.get_preds(dl=tst_dl)
aa['rating'] = preds

In [ ]:
aa.head()

,uid,item_id,rating
0,177489,601235,915.878601
1,14644,64855,915.878601
2,14699,64855,915.878601
3,190856,64855,915.878601
4,70394,64866,915.878601


In [ ]:
aa.describe()

In [ ]:
del f1,f2,t1,t2,nn_new

In [ ]:
rt = md['item_id'].unique()
len(rt)

1784

In [ ]:
ss_uid = []
ss_item = []
for i in ty[0:20000]:
  for j in rt:
    ss_uid.append(i)
    ss_item.append(j)

In [ ]:
df.item_id.nunique()

4837

In [ ]:
nn_new=pd.DataFrame()
nn_new['uid']=ss_uid
nn_new['item_id']=ss_item
del ss_uid
del ss_item
nn_new.head()

,uid,item_id
0,11,824963
1,11,818840
2,11,820362
3,11,55857
4,11,685639


In [ ]:
tst_dl = learn.dls.test_dl(nn_new)
preds,_=learn.get_preds(dl=tst_dl)
nn_new['rating'] = preds
del tst_dl,preds

In [ ]:
nn_new.head()

,uid,item_id,rating
0,11,824963,115.330620
1,11,818840,1.563821
2,11,820362,0.611922
3,11,55857,0.509533
4,11,685639,3.361279


In [ ]:
c_p = nn_new.set_index(['item_id']).groupby('uid')['rating'].nlargest(4).reset_index() 
del nn_new #avoid memory prb
c_p.head(10)

,uid,item_id,rating
0,11,155211,863.424805
1,11,84503,863.424805
2,11,92766,863.424805
3,11,70435,863.424805
4,15,155211,884.412292
5,15,84503,884.412292
6,15,92766,884.412292
7,15,70435,884.412292
8,18,155211,876.771179
9,18,84503,876.771179


In [ ]:
f11 = c_p[['uid','item_id']]

In [ ]:
f11.head()

,uid,item_id
0,11,155211
1,11,84503
2,11,92766
3,11,70435
4,15,155211


In [ ]:
df[df.item_id==84503]

,uid,item_id,rating


In [ ]:
del c_p

In [ ]:
f2 = pd.read_csv('/content/f2_20.csv')

In [ ]:
t11 = pd.concat([f11,f2],axis=0)
t11 = t11.reset_index(drop=True)
t11.shape

(165200, 2)

In [ ]:
rt=[25349, 55857, 20875, 145569]
ss_uid = []
ss_item = []
for i in ty_new:
  for j in rt:
    ss_uid.append(i)
    ss_item.append(j)
nn_new=pd.DataFrame()
nn_new['uid']=ss_uid
nn_new['item_id']=ss_item
del ss_uid #cleraring ram memory
del ss_item
nn_new.head(10)

,uid,item_id
0,14439,25349
1,14439,55857
2,14439,20875
3,14439,145569
4,14601,25349
5,14601,55857
6,14601,20875
7,14601,145569
8,79171,25349
9,79171,55857


In [ ]:
t22 = pd.concat([t11,nn_new],axis=0)
t22 = t22.reset_index(drop=True)
t22.shape

(169008, 2)

In [ ]:
t22.to_csv('very_ss.csv',index=False)

In [ ]:
nn_new=pd.DataFrame()
nn_new['uid']=ss_uid
nn_new['item_id']=ss_item
del ss_uid
del ss_item

In [ ]:
c_p = nn_new.set_index(['item_id']).groupby('uid')['rating_preds'].nlargest(4).reset_index() 
del nn_new
c_p.head()

In [ ]:
f1 = c_p[['uid','item_id']]
del c_p
f1.head(10)